In [78]:
import pandas as pd

#matplot
import matplotlib.pyplot as plt

import numpy as np

import os
import shutil

import sys
#glob
from glob import glob

데이터 출처

https://physionet.org/content/mimic-cxr-pe-severity/1.0.1/

In [ ]:
expert_data_df = pd.read_csv("/hdd/project/cylce_het/dataset/expert_report_edema_severity.csv")
consensus_data_df = pd.read_csv("/hdd/project/cylce_het/dataset/consensus_image_edema_severity.csv")

In [80]:
expert_data_df['consenus'] = 0
consensus_data_df['consenus'] = 1

In [81]:
expert_data_df

,subject_id,study_id,edema_severity,consenus
0,10040025,56874940,2,0
1,10040025,58793821,0,0
2,10152675,51914560,0,0
3,10152675,54401885,3,0
4,10213338,50988974,0,0
...,...,...,...,...
480,19757915,54693396,0,0
481,19757915,55134067,1,0
482,19757915,57527417,0,0
483,19804575,53050475,0,0


In [82]:
meta_data_df = pd.read_csv("/hdd/project/cylce_het/dataset/mimic_cxr_metadata_available_CHF_view.csv")

In [83]:
meta_data_df

,dicom_id,study_id,subject_id,gender,report_available,dicom_available,CHF,view
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,50414267,10000032,F,True,True,False,frontal
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,50414267,10000032,F,True,True,False,lateral
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,53189527,10000032,F,True,True,False,frontal
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,53189527,10000032,F,True,True,False,lateral
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,53911762,10000032,F,True,True,False,frontal
...,...,...,...,...,...,...,...,...
377092,428e2c18-5721d8f3-35a05001-36f3d080-9053b83c,57132437,19999733,F,True,True,False,frontal
377093,58c403aa-35ff8bd9-73e39f54-8dc9cc5d-e0ec3fa9,57132437,19999733,F,True,True,False,lateral
377094,58766883-376a15ce-3b323a28-6af950a0-16b793bd,55368167,19999987,F,True,True,False,frontal
377095,7ba273af-3d290f8d-e28d0ab4-484b7a86-7fc12b08,58621812,19999987,F,True,True,False,frontal


In [84]:
# Filter meta_data_df for rows where 'view' is 'frontal'
filtered_meta_data_df = meta_data_df[meta_data_df['view'] == 'frontal'].drop_duplicates(subset='study_id')

# Merge expert_data_df with the filtered meta_data_df
result_df = pd.merge(
    expert_data_df, 
    filtered_meta_data_df[['dicom_id', 'subject_id', 'study_id']], 
    on='study_id', 
    how='left', 
    suffixes=('', '_drop')  # Avoid suffixes for duplicate columns
)

# Drop unnecessary columns with "_drop" suffix
result_df = result_df.loc[:, ~result_df.columns.str.endswith('_drop')]


In [85]:
result_df

,subject_id,study_id,edema_severity,consenus,dicom_id
0,10040025,56874940,2,0,e82ef60c-a1777444-e0cec4c7-e6cbd366-e838760c
1,10040025,58793821,0,0,7d4a98c9-f3a75220-8a1e8f3d-e186582d-37d96bb5
2,10152675,51914560,0,0,3e9b02fb-3be41345-3907b0af-f35cfd1f-6fb31101
3,10152675,54401885,3,0,62395779-cc06e749-414bd87a-e664ecd4-cd2bb7b2
4,10213338,50988974,0,0,aaa2147e-bd1f19cb-f5e37785-7b524baa-d5b03b73
...,...,...,...,...,...
480,19757915,54693396,0,0,fa0c2ab8-4c8a6ee3-29f686bb-79365a4d-6cd9aab4
481,19757915,55134067,1,0,f100ec52-5afbc5bf-ecb3055a-3f3ef86c-c2cbc193
482,19757915,57527417,0,0,971d07bf-9ee17625-5b273248-8e1b17db-55ebe0fa
483,19804575,53050475,0,0,b04ec2a7-3669b1d5-429529cf-3d6585b9-a30d642d


In [86]:
expert_data_df

,subject_id,study_id,edema_severity,consenus
0,10040025,56874940,2,0
1,10040025,58793821,0,0
2,10152675,51914560,0,0
3,10152675,54401885,3,0
4,10213338,50988974,0,0
...,...,...,...,...
480,19757915,54693396,0,0
481,19757915,55134067,1,0
482,19757915,57527417,0,0
483,19804575,53050475,0,0


In [87]:
consensus_data_df

,subject_id,study_id,dicom_id,edema_severity,consenus
0,10062617,55170181,5b8f4e5f-074a3958-ca8e7fc2-100ffa07-6f553e72,0,1
1,10216097,50944321,f1d29d45-c7724009-cdf6d1e2-c5d892cc-ced1a59d,1,1
2,10216097,58218519,bb2e426d-fa0792ea-d460f5fa-c4932cbd-91e5328e,0,1
3,10228186,57118202,621d8f8d-5a467b27-77152e93-31311462-b5921953,0,1
4,10313763,51527697,eee23a20-30e4d811-4ddcd3f7-8637280e-b6a719d7,1,1
...,...,...,...,...,...
136,19735078,50027115,0e9e1dfc-73911989-41b4eb1c-68795b5c-712f3e6e,1,1
137,19783125,55253826,e12ad3be-5d838a17-7b7f1f2c-111b45f6-b19b5552,1,1
138,19792891,57548918,0f37b602-ac1b619c-78f78946-82bbceb3-0fa34266,0,1
139,19822698,52549668,1c4775e4-bfbcfb51-2135812e-cbb7c2a5-750af140,0,1


In [88]:
concated_df = pd.concat([result_df, consensus_data_df], axis=0)

In [89]:
concated_df

,subject_id,study_id,edema_severity,consenus,dicom_id
0,10040025,56874940,2,0,e82ef60c-a1777444-e0cec4c7-e6cbd366-e838760c
1,10040025,58793821,0,0,7d4a98c9-f3a75220-8a1e8f3d-e186582d-37d96bb5
2,10152675,51914560,0,0,3e9b02fb-3be41345-3907b0af-f35cfd1f-6fb31101
3,10152675,54401885,3,0,62395779-cc06e749-414bd87a-e664ecd4-cd2bb7b2
4,10213338,50988974,0,0,aaa2147e-bd1f19cb-f5e37785-7b524baa-d5b03b73
...,...,...,...,...,...
136,19735078,50027115,1,1,0e9e1dfc-73911989-41b4eb1c-68795b5c-712f3e6e
137,19783125,55253826,1,1,e12ad3be-5d838a17-7b7f1f2c-111b45f6-b19b5552
138,19792891,57548918,0,1,0f37b602-ac1b619c-78f78946-82bbceb3-0fa34266
139,19822698,52549668,0,1,1c4775e4-bfbcfb51-2135812e-cbb7c2a5-750af140


In [65]:
file_list = glob("/hdd/project/cxr_haziness/data/mimic/processed_images/*.png")

In [77]:
def get_dicom_id(file_path):
    return os.path.basename(file_path).split(".")[0]

def get_file_path(dicom_id):
    return f"/hdd/project/cxr_haziness/data/mimic/processed_images/{dicom_id}.png"





In [76]:


def file_move(data_df, file_id_list):
    move_dir = "/hdd/project/cylce_het/dataset/edema_raw_images/"
    for idx, row in data_df.iterrows():
        dicom_id = row["dicom_id"]
        if dicom_id in file_id_list:
            file_path = get_file_path(dicom_id)
            shutil.copy(file_path, move_dir)
            #print(dicom_id)
        else:
            print(f"{dicom_id} not found")
            
file_id_list = [get_dicom_id(file_path) for file_path in file_list]
file_move(concated_df,file_id_list)

In [90]:
# data_df 출력
concated_df.to_csv("/hdd/project/cylce_het/dataset/edema_severity.csv", index=False)